### Real Options Valuation and Monte Carlo simulation, DM and FPOM.

**This is a sheet for working with all three cases investigated in Santeri's Master thesis on Real Options.**

Combining previously modeled calculations into a single Python-enabled Jupyter notebook. Generalizing, modularizing and improving outputs.

**Note:** To avoid errors ensure all notebook cells are executed sequentially. 

### Setup

Implemented using the Conda kernel. Python version 3.9.18

In [23]:
# Import necessary libraries
import numpy as np
import pandas as pd


# Model and declare initial application state
case_data = {
    "case_npvs": np.array([]),
    "additional_arrays": {
        "launch_costs_array": np.array([]),
        "operating_inflows_array": np.array([]),
        "operating_outflows_array": np.array([]),
    },
    "inputs": {}
}


### Utilities

This section implements some of the necessary utility functions used later in this notebook.

In [19]:
# This function helps form total discounted values for projects out of yearly cash flow values before simulation
def calculate_discounted_values(values_dict, risk_rate, start_period, periods_in_operation):
    # If the passed in values are launch costs, then we spread the that and discount it only for the years before launch.
    return {k: sum(v / (1 + risk_rate) ** period for period in range(start_period + 1, start_period + 1 + periods_in_operation)) for k, v in values_dict.items()}

### Import case data

Supports simple copy-pasting for long spreadsheet rows. All three cases are imported below.

In [20]:
case_data["inputs"]["start_operating_after_periods"] = 2 # After this amount of years operation starts
case_data["inputs"]["periods_in_operation"] = 50 # Years of operation when launched and operating
case_data["inputs"]["operating_risk_rate"] = 0.07 # Risk, hurdle or discount rate for operating cash flows
case_data["inputs"]["launch_cost_risk_rate"] = 0.03 # Risk, hurdle or discount rate for launch costs
case_data["inputs"]["yearly_op_revenues_range"] = {"pess": 7.5, "base": 9.5, "opti": 10.5} # Range of yearly operating revenues -> pess: pessimistic, base: base case, opti: optimistic
case_data["inputs"]["yearly_op_cost_range"] = {"pess": 1.5, "base": 1.14, "opti": 0.84} # Range of yearly operating costs -> pess: pessimistic, base: base case, opti: optimistic
case_data["inputs"]["total_launch_cost_range"] = {"pess": 40, "base": 35, "opti": 30} # Range of total launch costs -> pess: pessimistic, base: base case, opti: optimistic

### Perform the Monte Carlo simulation

Defines the simulation and proceeds to run it for all three business cases.

In [22]:
# Define number of simulation runs
num_simulations = 100000

# Create a function to simulate a single draw or sample from a triangular distribution
def simulation_single_sample(pess, base, opti):
    # Logic to handle drawing the triangular distribution correctly
    low, mode, high = sorted([pess, base, opti])
    sample = np.random.triangular(low, mode, high)
    return sample


# Create a function for running the Monte Carlo simulation
def monte_carlo_simulation(launch_cost, total_op_revenues, total_op_costs, num_simulations):
    npvs = []
    launch_costs_array = []
    operating_inflows_array = []
    operating_outflows_array = []
    average_elec_prices = []
    storage_capacity = []

    for _ in range(num_simulations):

        # Discounting on every simulation run
        case_data["inputs"]["total_op_revenues"] = calculate_discounted_values(case_data["yearly_op_revenues_range"], case_data["operating_risk_rate"], case_data["start_operating_after_periods"], case_data["periods_in_operation"])
        case_data["inputs"]["total_op_costs"] = calculate_discounted_values(case_data["yearly_op_cost_range"], case_data["operating_risk_rate"], case_data["start_operating_after_periods"], case_data["periods_in_operation"])
        case_data["inputs"]["launch_cost"] = {k: sum((v / case_data["start_operating_after_periods"]) / (1 + case_data["launch_cost_risk_rate"]) ** period for period in range(1, case_data["start_operating_after_periods"] + 1)) for k, v in case_data["total_launch_cost_range"].items()}

        simul_launch_cost = simulation_single_sample(launch_cost["pess"], launch_cost["base"], launch_cost["opti"])
        simul_operating_inflows = simulation_single_sample(total_op_revenues["pess"], total_op_revenues["base"], total_op_revenues["opti"])
        simul_operating_outflows = simulation_single_sample(total_op_costs["pess"], total_op_costs["base"], total_op_costs["opti"])

        # Out of all investment inflows substract its launch cost and total operating outflows
        npv = simul_operating_inflows - (simul_launch_cost + simul_operating_outflows)

        launch_costs_array.append(simul_launch_cost)
        operating_inflows_array.append(simul_operating_inflows)
        operating_outflows_array.append(simul_operating_outflows)
        npvs.append(npv)
    return npvs, launch_costs_array, operating_inflows_array, operating_outflows_array

# Run simulations and store results each case's arrays
case_data["case_npvs"], case_data["additional_arrays"]["launch_costs_array"], case_data["additional_arrays"]["operating_inflows_array"], case_data["additional_arrays"]["operating_outflows_array"] = monte_carlo_simulation(case_data["inputs"]["launch_cost"], case_data["inputs"]["total_op_revenues"], case_data["inputs"]["total_op_costs"], num_simulations)

KeyError: 'yearly_op_revenues_range'

### Exporting the data for Mariia

In [38]:
df = pd.DataFrame({
    'npvs': case_data[2]["case_npvs"],
    'launch_costs': case_data[2]["additional_arrays"]["launch_costs_array"],
    'operating_inflows': case_data[2]["additional_arrays"]["operating_inflows_array"],
    'operating_outflows': case_data[2]["additional_arrays"]["operating_outflows_array"]
})
df.to_csv('specialized_case_data.csv', index=False)